<a href="https://colab.research.google.com/github/Kasper-Raupach-Haurum/M6-Data-Engineering-and-MLOps/blob/main/M6_Data_Engineering_and_MLOps_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In conclusion, our Assignment 1 notebook demonstrates the implementation of a semantic search application using Gradio, SQLite, and Hugging Face's Sentence Transformers. 

By creating & connecting to an SQLite database containing a limited set of documents, we preprocess and create embeddings for each document using a pretrained SentenceTransformer("paraphrase-MiniLM-L6-v2") model.

The user can then search for similar documents by entering a query through the Gradio interface, which displays the most relevant results based on cosine similarity scores. 

Finally, we enhance the application by incorporating a more comprehensive dataset, the 20Newsgroup dataset from scikit-learn, to showcase the potential of semantic search in finding relevant information across a larger collection of documents.